In [2]:
#You must include the following in your 
#%run "../PyUtils/common.ipynb"
import os;

import openpyxl
import string
from openpyxl.utils import *; 

#(_get_column_letter, column_index_from_string)
#openpyxl.utils.get_column_interval(34) # returns 'AH'
#column_index_from_string('AH') #returns 34
#http://openpyxl.readthedocs.io/en/2.3.3/_modules/openpyxl/utils.html
#ws['C3'].value
#for i in openpyxl.utils.cols_from_range('A1:B3'): print (i)    
#for i in openpyxl.utils.rows_from_range('A1:B3'): print (i)

In [3]:
toolbarJS = '''
<script type="text/Javascript">
function callback(out_type, out_data){
    // do_something
}
function callsave(v){
    var kernel = IPython.notebook.kernel;

    command = "saveDF("+v+")"
    console.log("Executing Command: " + command);
    g = kernel.execute(command);
    console.log(g)
}
             
function callsave1(tname, df){
    var kernel = Jupyter.notebook.kernel;
    var ncols = $("#t1 > tbody > tr:first > th").length
    var cnt = 0
    $("#t1 td").each(function (i, j ){
        var td  = $(this)
        var ct  = $(this).text()
        var c   = $(this).index();
        var r   = $(this).closest('tr').index();
        
        var realColumnIndex = 0;
        $(this).prevAll("td").each(function() {
            realColumnIndex += this.colSpan;
        });
        
        command = df+'.loc['+r+']['+realColumnIndex+']= "' + td.text() + '";'
        //console.log( i, j, td.text() , command)
        if ( cnt++ > 0) {
            console.log( i, r,realColumnIndex, ct, td.attr('colspan') )
            //console.log("Executing Command: " + command);
            g = kernel.execute(command);
        }
        else {
            return;
        }
    } )
           
}         
</script>

'''

toolbar = '''
<div style="background-color:#f7f7f7; border: 1px dotted #cccccc; width:100%; padding:0px; margin: 0px">
<button onclick="callsave1('#t1', 'df2')">Save to DataFrame</button>
<label id='STATUS_#HTML_t1' style="border: 0px solid black">status</label>
</div>
<div >
<table   width=100% style="border: 0px;">
<tr style="border: 0px;border-left: 0px solid gray;">

<td id='CELLADDRESS_#HTML_t1' contenteditable style="border: 0px;border-right: 1px dotted gray;width: 90px"> </td>

<td style="border: 0px;text-align:center;" width=80px>
<font style="font-family: serif; font-weight: 900; font-size: 16px; font-style: ooblique;color:#cccccc; text-align:center">
&#10005 &nbsp;&nbsp;&nbsp;&#10003&nbsp;&nbsp;&nbsp; <i>f<sub>x</sub> </i>
</font>
</td>

<td style="border:0px; border-left: 1px solid gray;width: 1px"></td>

<td id='CELLContents_#HTML_t1' contenteditable style="border: 0px;border-right: 5px solid gray;">Blah  </td></tr>

</table>
</div>
'''

In [5]:
jjs = '''
<script>
console.clear();
var getcell = function(table, row,col) { return $( table + ' tr:eq('+row+') > td:eq(' + col +')')}
var removeCell = function(table, row,col) { 
    var c = getcell(table, row,col)
    c.remove();
}

var o="";
$("#t1 tr td").each(function () {
    var ty = $(this).text()
    o += ty
});

var b=""
//var rHeights=  []
//var cWidths =  []

$("#t1 tr").each(function (i) {
    var tr = $(this)
    var nh = rHeights[i] + 'px'
    //tr.css('height', nh);
});

$("#t1 th").each(function (i) {
    var th = $(this)
    var nh = cWidths[i] + 'px'
    th.css("background-color", '#ededed');
    th.css("text-align", 'center');
    //console.log(nh)
});

$("#t1 tr:first th").each(function (i, j, k) {
    var tr = $(this)
    var nh = cWidths[i] + 'px'
    tr.css("background-color", '#f0f0f0');
    tr.css("overflow", 'hidden');
    //tr.css("width", nh);
    //console.log(tr.text() + " " + nh)
});

$("#t1 th").resizable()

$("#t1 tr:first th:first").css('width', '30px')
var v = $("#t1 tr:first th").css('width')
$("#idtest1").css('width', v)


//------------------------------------
// Column changing
//
var cells = $("#t1").find('td');
//console.log(cells)
cells.each(function () {    
    $(this).focusout(function() {
        var ct = $(this).text()
        var nt = $(this).data("original")
        if ( ct != nt ) {
            console.log( ct + " !=" + nt);
            $('#STATUS_t1').text(ct +" : " + nt)
            
        }
    }); 
    
    $(this).focus(function(){
        var ct  = $(this).text()
        var col = $(this).index();
        var row = $(this).closest('tr').index();
        
        if (formulas[row] != undefined && formulas[row][col-1] != undefined){
            ct = formulas[row][col-1] + "  "
        }
        
        $(this).data("original", ct)
        var nt = $(this).data("original")
        var on = (ct+ ":" + nt)
        $('#CELLContents_t1').text(ct)
        $('#CELLADDRESS_t1').text(row +"," + col)
        
        console.log(on + " " + row + " " + col)
    });  
});

</script>
''';



def openFileAndShow( file="test.xlsx", tname= 't1', sheetname=0):
    df2 = LoadDataSet(file, headers=None, sheetname=sheetname);
    colNames = [c for c in  string.ascii_uppercase[0:df2.shape[1]] ]
    df2.columns = (colNames)

    df2.fillna('', inplace=True)
    df2.index = range(len(df2))
    ht = df2.to_html()
    ht = ht.replace("<table ", "<table id='{}' width='100%' ".format(tname))
    ht = ht.replace("<td", "<td contenteditable ")
    #ht = ht.replace("<td", "<td ")
    
    wb,ws,genjs = getStyles(file, sheetname, df2)
    
    mergeJS = mergeCells(ws, tname)
    
    fjs = (genjs)+ jjs+ mergeJS + toolbarJS
    
    html = toolbar +"<div>" + ht + "</div>" + fjs
    html = html.replace('#t1', "#"+tname) 
    html = html.replace('#HTML_t1', tname) 
    
    display(HTML(html))
    #display(HTML(fjs) )
    return wb, ws, html, df2
    
def getFormulas(ws, df2):
    formulas= defaultdict(str);

    rows = df2.shape[0]
    cols = df2.shape[1]    
    for i in range(rows):
        for j in range(cols):
            ii = i+1
            jj = j+1
                
            c= ws.cell(row=ii, column=jj);
            v = c.value;
            
            if ( v is not None and type(v) == str and v.strip().startswith('=') ):
                formulas[(ii,jj)] = v
    return formulas

def getStyles(file, sheetname=0, df2=None, wb=None):
    
    formulas= "var formulas=[];"
    genjs='<script>'

    wb = wb if wb is not None else openpyxl.load_workbook(file)
    sh=[]
    for c in wb.get_sheet_names():
        sh.append(wb[c])

    ws=sh[sheetname]
    rh = [];

    numRows = min(len(ws.row_dimensions), df2.shape[0])
    for c in range(1,numRows):
        rh.append(ws.row_dimensions[c].height  )

    maxC = 0;
    for c in ws.column_dimensions:
        cidx = column_index_from_string(c)
        maxC = max(maxC, cidx)

    cw = [3]
    for cc in range(1,maxC):
        cn = openpyxl.utils.get_column_letter(cc)
        if (cn in ws.column_dimensions):
            cw.append(ws.column_dimensions[cn].width)
        else:
            cw.append(10)

    genjs += 'var rHeights= {}'.format(rh) + "\n"
    genjs += 'var cWidths = {}'.format(cw)  + "\n"

    rows = df2.shape[0]
    cols = df2.shape[1]
    

    #Find the most common one
    common = defaultdict(int);
    for i in range(rows):
        for j in range(cols):
            c= ws.cell(row=i+1, column=j+1);
            f = c.font
            fontColor = '' if (str(f.color.rgb).startswith('Value')) else "#"+str(f.color.rgb)[2:];
            val = "{}--{}--{}--{}".format(c.font.name, c.font.b, c.font.sz, fontColor)
            common[val] +=1
        
    maxkey = max(common.keys(), key=lambda k: common[k])
    
    for i in range(rows):
        for j in range(cols):
            ii = i+1
            jj = j+1
                
            c= ws.cell(row=ii, column=jj);
            v = c.value;
            
            if ( v is not None and type(v) == str and v.strip().startswith('=') ):
                formulas += "if (formulas[{}] == undefined ) formulas[{}]=[]; formulas[{}][{}]='{}'\n".format(i,i,i,j,v)
            
            
            # Style information below
            val = "{}--{}--{}--{}".format(c.font.name, c.font.b, c.font.sz, fontColor)
            if ( val == maxkey):
                pass; #continue
                # For now we are not doing anything - but we can set this as default for the table
                # and reduce java script length - more later
            f  = c.font
            fontColor = '' if (str(f.color.rgb).startswith('Value')) else "#"+str(f.color.rgb)[2:];
            bgColor   = 'lime' if (str(c.fill.fgColor.rgb).startswith('Value')) else "#"+str(c.fill.fgColor.rgb)[2:];
            border = '0px' if (c.border is None or len(str(c.border)) <= 0) else ''
            if c.font.b:
                genjs +=("$('#t1 tr:eq({}) td:eq({})').css('font-weight','bold' )".format(ii,j) ) + "\n"
                
            if c.font.i:
                itl=("$('#t1 tr:eq({}) td:eq({})').css('font-style','italic' )".format(ii,j) ) + "\n"
                genjs += itl
            
                
            if ( not bgColor.startswith("#000000")):
                genjs += ("$('#t1 tr:eq({}) td:eq({})').css('background-color','{}' )//{}".format(ii,j, bgColor, v) ) + "\n"


            if(c.alignment.horizontal is not None):
                genjs +=("$('#t1 tr:eq({}) td:eq({})').css('text-align','{}' )".format(ii,j, c.alignment.horizontal)  ) + "\n"

            #print (i, j, c.font.name, c.font.b, fontColor, bgColor, c.font.sz, border, '-', sep=" : ") + "\n"
            genjs +=("$('#t1 tr:eq({}) td:eq({})').css('font','{}' )".format(ii,j, c.font.name)  ) + "\n"
            genjs +=("$('#t1 tr:eq({}) td:eq({})').css('color','{}' )".format(ii,j, fontColor) ) + "\n"
            genjs +=("$('#t1 tr:eq({}) td:eq({})').css('fontSize','{}px' )".format(ii,j, c.font.sz) ) + "\n"
            genjs +=("$('#t1 tr:eq({}) td:eq({})').css('border','{}' )".format(ii,j, border) ) + "\n"
    genjs += "\n" + formulas + "</script>";
    return wb, ws, genjs

def mergeCells(ws, tabn='#t1'):
    remv ={}
    merg =""
    tabn = tabn if tabn.startswith("#") else "#"+tabn; 
    for r in ws.merged_cell_ranges:
        mc = list(openpyxl.utils.rows_from_range(r))
        firstCell = mc[0][0]
        rf = openpyxl.utils.coordinate_to_tuple(firstCell)
        rowSpan = len(mc)
        colSpan = len(mc[0])

        merg += "var c = getcell('{}', {}, {}) /*{}*/; c.attr('rowspan',{}); c.attr('colspan', {})\n".format(tabn, rf[0], rf[1]-1, firstCell, rowSpan, colSpan)
        firstOne = True;
        for row in mc:
            for idx in row:
                if ( firstOne ):
                    firstOne = False;
                    continue;
                rf = openpyxl.utils.coordinate_to_tuple(idx)
                rem = '[{}, {}]'.format(rf[0], rf[1]-1)
                remv[rf] = rem
    
    remCells = ""
    for c in sorted(remv.keys(), reverse=True):
        remCells += "removeCell('{}', {}, {})\n".format(tabn, c[0], c[1]-1) 
   
    ret = "<script>\n{}\n{}\n</script>".format(merg, remCells)
    return ret;


SyntaxError: invalid syntax (<ipython-input-5-2b7655908f46>, line 141)